In [1]:
proj_path = '/home/ajhnam/projects/hidden_singles_public/'

In [2]:
import sys
sys.path.append(proj_path + 'python/')

import random
import numpy as np
import itertools
import pandas as pd
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader as DataLoader
from tqdm.auto import tqdm

from hiddensingles.misc import torch_utils as tu
from hiddensingles.misc import utils, TensorDict, TensorDictDataset, RRN
from hiddensingles.sudoku.grid import GridString

In [3]:
device = 3

In [4]:
def flip_seed_puzzles(df, op):
    """
    Flips puzzles and solutions of all puzzles and solutions in the dataframe
    """
    df = df.copy()
    puzzles_flip = [flip_gridstring(gs, op) for gs in df.puzzle]
    solutions_flip = [flip_gridstring(gs, op) for gs in df.solution]
    puzzles_flip, solutions_flip = zip(*[make_seed(p, s) for p, s in zip(puzzles_flip, solutions_flip)])
    df[['puzzle', 'solution']] = np.array([puzzles_flip, solutions_flip]).T
    return df

def flip_gridstring(gridstring: str, op: str):
    assert op in ('lr', 'ud', 'both')
    
    gridstring = GridString.load(gridstring)
    array = gridstring.array()
    
    if op == 'lr':
        array = np.fliplr(array)
    elif op == 'ud':
        array = np.flipud(array)
    else:
        array = np.flip(array)
        
    gridstring = GridString.load_array(gridstring.dim_x, gridstring.dim_y, array)
    return str(gridstring)

def map_digits(puzzle_gs: str, solution_gs: str, mapping: dict):
    puzzle_gs = GridString.load(puzzle_gs).map_digits(mapping)
    solution_gs = GridString.load(solution_gs).map_digits(mapping)
    return str(puzzle_gs), str(solution_gs)

def make_seed(puzzle_gs: str, solution_gs: str):
    solution_gs = GridString.load(solution_gs)
    mapping = solution_gs.seed_mapping()
    
    solution_gs = solution_gs.map_digits(mapping)
    puzzle_gs = GridString.load(puzzle_gs).map_digits(mapping)
    return str(puzzle_gs), str(solution_gs)

def random_map(puzzle_gs: str, solution_gs: str):
    puzzle = GridString.load(puzzle_gs)
    solution = GridString.load(solution_gs)
    
    mapping = {i: j for i, j in zip(range(1, puzzle.max_digit), 1 + np.random.permutation(puzzle.max_digit))}
    puzzle = puzzle.map_digits(mapping)
    solution = solution.map_digits(mapping)

    return str(puzzle), str(solution)

def random_map_df(df):
    """
    Applies random_map to all puzzles and solutions in the dataframe
    """
    df = df.copy()
    puzzles, solutions = zip(*[random_map(p, s) for p, s in zip(df.puzzle, df.solution)])
    df[['puzzle', 'solution']] = np.array([puzzles, solutions]).T
    return df

In [5]:
def get_results(model, dataset, batch_size, num_steps=16, optimizer=None):
    train = optimizer is not None
    
    dataloader = DataLoader(TensorDictDataset(dataset), batch_size=batch_size, shuffle=train)
    
    losses = []
    correct = []
    for dset in dataloader:
        dset = TensorDict(**dset)
        
        if train:
            optimizer.zero_grad()
            outputs = model(dset.inputs, num_steps=num_steps)
        else:
            with torch.no_grad():
                outputs = model(dset.inputs, num_steps=num_steps)
        outputs = outputs.view(-1, num_steps, model.max_digit, model.max_digit, model.max_digit)
        targets = tu.expand_along_dim(dset.targets, 1, num_steps)
        loss = tu.cross_entropy(outputs, targets)
        
        if train:
            loss.backward()
            optimizer.step()
        
        # record
        losses.append(loss.item())
        correct.append((outputs.argmax(-1) == targets)[:,-1])
    
    correct = torch.cat(correct)
    loss = torch.tensor(losses).mean()
    accuracy = correct.float().mean().cpu()
    solved = correct.all(-1).all(-1).float().mean().cpu()
    
    results = TensorDict(loss=loss,
                         accuracy=accuracy,
                         solved=solved)
    return results

In [6]:
num_sample_multiplier = 2 # Creates 800*N training samples and 200*N test samples

df_puzzles = pd.read_csv(proj_path + '/data/2x3_puzzle_seeds.tsv', sep='\t')

# Select only puzzles with 17 hints, remove puzzles with same solutions
df_puzzles = df_puzzles[(df_puzzles.num_hints == 17)].drop_duplicates('solution').reset_index(drop=True)

# Create flipped versions of the same puzzles
df_puzzles['seed_sol'] = df_puzzles.solution
df_flip = flip_seed_puzzles(df_puzzles, 'both')
df_fliplr = flip_seed_puzzles(df_puzzles, 'lr')
df_flipud = flip_seed_puzzles(df_puzzles, 'ud')
df_puzzles = pd.concat([df_puzzles, df_flip, df_fliplr, df_flipud]).drop_duplicates('solution')

# Only choose seed solutions where all 4 orientations exist
counts = df_puzzles.groupby('seed_sol').count()
seed_sols = counts[counts.puzzle == 4].reset_index()[['seed_sol']].head(250)
seed_sols['train'] = [i < 200 for i in range(250)]
df_puzzles = df_puzzles.merge(seed_sols)
df_puzzles = df_puzzles[['train', 'puzzle', 'solution']]

# Create multiple versions of same puzzles by shuffling digits
df_puzzles = pd.concat([random_map_df(df_puzzles) for i in range(num_sample_multiplier)]).reset_index(drop=True)

In [7]:
# Turn puzzles into tensors

inputs = torch.tensor([GridString.load(p).array() for p in df_puzzles[df_puzzles.train].puzzle])
targets = -1 + torch.tensor([GridString.load(p).array() for p in df_puzzles[df_puzzles.train].solution])
train_dset = TensorDict(inputs=inputs, targets=targets).to(device)
inputs = torch.tensor([GridString.load(p).array() for p in df_puzzles[~df_puzzles.train].puzzle])
targets = -1 + torch.tensor([GridString.load(p).array() for p in df_puzzles[~df_puzzles.train].solution])
test_dset = TensorDict(inputs=inputs, targets=targets).to(device)

In [8]:
model = RRN(dim_x=2,
            dim_y=3,
            digit_embed_size=7,
            num_mlp_layers=0,
            hidden_vector_size=48,
            message_size=48,
            encode_coordinates=False).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [9]:
num_steps = 16
batch_size = 400
num_epochs = 500
print_epochs = 10

tr_result = get_results(model, train_dset, batch_size=batch_size, num_steps=num_steps)
te_result = get_results(model, test_dset, batch_size=batch_size, num_steps=num_steps)
tr_results = [tr_result]
te_results = [te_result]

for epoch in tqdm(range(num_epochs)):
    tr_result = get_results(model, train_dset, batch_size=batch_size, num_steps=num_steps, optimizer=optimizer)
    tr_results.append(tr_result)
    te_result = get_results(model, test_dset, batch_size=batch_size, num_steps=num_steps)
    te_results.append(te_result)

    if epoch % print_epochs == 0:
        utils.kv_print(epoch=epoch, loss=tr_result.loss,
                       tr_acc=tr_result.accuracy, tr_sol=tr_result.solved,
                       te_acc=te_result.accuracy, te_sol=te_result.solved)
        
tr_results = TensorDict.stack(tr_results, 0)
te_results = TensorDict.stack(te_results, 0)

epoch: 0 | loss: 1.746 | tr_acc: 0.303 | tr_sol: 0.0 | te_acc: 0.301 | te_sol: 0.0
epoch: 10 | loss: 1.04 | tr_acc: 0.696 | tr_sol: 0.0 | te_acc: 0.7 | te_sol: 0.0
epoch: 20 | loss: 0.722 | tr_acc: 0.808 | tr_sol: 0.001 | te_acc: 0.814 | te_sol: 0.0
epoch: 30 | loss: 0.561 | tr_acc: 0.861 | tr_sol: 0.008 | te_acc: 0.855 | te_sol: 0.015
epoch: 40 | loss: 0.44 | tr_acc: 0.914 | tr_sol: 0.102 | te_acc: 0.896 | te_sol: 0.052
epoch: 50 | loss: 0.38 | tr_acc: 0.929 | tr_sol: 0.169 | te_acc: 0.917 | te_sol: 0.107
epoch: 60 | loss: 0.344 | tr_acc: 0.94 | tr_sol: 0.244 | te_acc: 0.925 | te_sol: 0.16
epoch: 70 | loss: 0.316 | tr_acc: 0.946 | tr_sol: 0.282 | te_acc: 0.927 | te_sol: 0.212
epoch: 80 | loss: 0.284 | tr_acc: 0.959 | tr_sol: 0.414 | te_acc: 0.94 | te_sol: 0.34
epoch: 90 | loss: 0.262 | tr_acc: 0.967 | tr_sol: 0.507 | te_acc: 0.95 | te_sol: 0.415
epoch: 100 | loss: 0.242 | tr_acc: 0.973 | tr_sol: 0.589 | te_acc: 0.956 | te_sol: 0.477
epoch: 110 | loss: 0.222 | tr_acc: 0.981 | tr_sol: 0

In [10]:
tr_df = tr_results.to_dataframe({0: 'epoch'})
tr_df['dataset'] = 'train'
te_df = te_results.to_dataframe({0: 'epoch'})
te_df['dataset'] = 'test'
df = pd.concat([tr_df, te_df])

df.to_csv(proj_path + "data/rrn/sudoku_2x3_results.tsv", sep='\t', index=False)